In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!nvidia-smi

Mon Dec 11 05:20:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install --quiet git+https://github.com/huggingface/transformers sentencepiece

In [4]:
from transformers import SeamlessM4TModel

model = SeamlessM4TModel.from_pretrained("facebook/hf-seamless-m4t-medium")
# model = SeamlessM4TModel.from_pretrained("/content/drive/MyDrive/NLP/text2speech/Seamlessm4t/seamlessm4t-medium")

/opt/conda/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


config.json:   0%|          | 0.00/2.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/5.35k [00:00<?, ?B/s]

In [5]:
#SAVE THE PRETRAINED MODEL
model.save_pretrained("/kaggle/working/")


In [6]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [7]:
print(device)

cuda:0


In [8]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("facebook/hf-seamless-m4t-medium")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


preprocessor_config.json:   0%|          | 0.00/3.36k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.33k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.29k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
processor.save_pretrained( "/kaggle/working/")

In [20]:
text1="""This article has garnered numerous comments, consistently\
leaning towards the negative. These comments are from individuals who either failed\
to comprehend the content or were expecting another article featuring a “Get Rich Quick” \
strategy, which is not the focus of this piece. If your intention is not to carefully read \
the article and understand its implications, please refrain from proceeding further."""

text2="Hello, This is Samuel, Working in Kaggle. Here we are all there to interview you man!!"
text_inputs = processor(text = text1, src_lang="eng", return_tensors="pt").to(device)

In [21]:
audio_array_from_text = model.generate(**text_inputs, tgt_lang="eng")[0].cpu().numpy().squeeze()
# audio_array_from_audio = model.generate(**audio_inputs, tgt_lang="rus")[0].cpu().numpy().squeeze()

In [22]:
audio_array_from_text

array([1.07686734e-04, 9.92998248e-05, 9.45068896e-05, ...,
       3.59498663e-05, 6.95954077e-05, 8.87444476e-05], dtype=float32)

In [23]:
#This is in Russia
from IPython.display import Audio

sample_rate = model.config.sampling_rate
Audio(audio_array_from_text, rate=sample_rate)

**Testing "Zephyr-7B-β" model for text generation**

In [24]:
# Install transformers from source - only needed for versions <= v4.34
# pip install git+https://github.com/huggingface/transformers.git
# pip install accelerate

import time
%%time
import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])
# <|system|>
# You are a friendly chatbot who always responds in the style of a pirate.</s>
# <|user|>
# How many helicopters can a human eat in one sitting?</s>
# <|assistant|>
# Ah, me hearty matey! But yer question be a puzzler! A human cannot eat a helicopter in one sitting, as helicopters are not edible. They be made of metal, plastic, and other materials, not food!


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
Matey, I'm afraid there's no answer to your question as humans cannot eat helicopters. Helicopters are not a form of food, they're a mode of transportation. I'm afraid you've been swallowin' too many grog and hallucinatin' about eatin' copters!


In [25]:
print(outputs[0]["generated_text"])

<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
Matey, I'm afraid there's no answer to your question as humans cannot eat helicopters. Helicopters are not a form of food, they're a mode of transportation. I'm afraid you've been swallowin' too many grog and hallucinatin' about eatin' copters!


In [28]:
import time
%%time
context = """
this brief article, I showcase an options trading strategy with a Probability of Profit (PoP) of 100%.\
And this is not really surprising, as you, my esteemed reader, will discover in the following paragraphs.\
This possibility becomes quite obvious when you understand the conditions necessary to achieve such a remarkable outcome.

What is truly important in this article is to showcase another feature of my Python library, OptionLab, which\
brings a lot of flexibility to those studying options trading from a quantitative perspective."""
messages = [
    {
        "role": "system",
        "content": f"""As your task is to extract the answer from the given\
      context which is delimited by triple backticks: ```{context}``` """,
    },
    {"role": "user", "content": "What is the context about?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

<|system|>
As your task is to extract the answer from the given      context which is delimited by triple backticks: ```
this brief article, I showcase an options trading strategy with a Probability of Profit (PoP) of 100%.And this is not really surprising, as you, my esteemed reader, will discover in the following paragraphs.This possibility becomes quite obvious when you understand the conditions necessary to achieve such a remarkable outcome.

What is truly important in this article is to showcase another feature of my Python library, OptionLab, whichbrings a lot of flexibility to those studying options trading from a quantitative perspective.```</s>
<|user|>
What is the context about?</s>
<|assistant|>
The context is about introducing a high-probability options trading strategy using the Python library OptionLab from the author's own experience. The author emphasizes that this strategy has a Probability of Profit (PoP) of 100%, making it a remarkable outcome. The author further exp

In [29]:
prompt

'<|system|>\nAs your task is to extract the answer from the given      context which is delimited by triple backticks: ```\nthis brief article, I showcase an options trading strategy with a Probability of Profit (PoP) of 100%.And this is not really surprising, as you, my esteemed reader, will discover in the following paragraphs.This possibility becomes quite obvious when you understand the conditions necessary to achieve such a remarkable outcome.\n\nWhat is truly important in this article is to showcase another feature of my Python library, OptionLab, whichbrings a lot of flexibility to those studying options trading from a quantitative perspective.```</s>\n<|user|>\nWhat is the context about?</s>\n<|assistant|>\n'